In [1]:
import sklearn
from sklearn import datasets
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np 
import torch.optim as optim 
import sys 
import os 
import matplotlib.pyplot as plt

In [2]:
project_basedir=os.path.join('..','..','..')
project_basedir = os.path.abspath(project_basedir)
sys.path.insert(0,project_basedir)

In [3]:
from core.np.datasets.IrisDataset import Iris 
import core.np.Nodes as node 
from core import debug, info, log_at_info
import core.np.Nodes as node
import core.np.Loss as loss
import core.np.Activations as act
from  core.np.utils import to_one_hot, FilePersistenceHelper

In [4]:
iris = Iris()

# Multilayer 

In [5]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.dense1 = nn.Linear(4,16)
        self.dense2 = nn.Linear(16,10)
        self.dense3 = nn.Linear(10,3)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = torch.tanh( self.dense1(x)) 
        x = F.relu(self.dense2(x))
        x = self.dense3(x) 
        x = self.softmax(x) 
        return x 
    
    def _set_weights(self, layer, w, b):
        if w is None:
            print("Null w")
        wt = torch.from_numpy(w).float() 
        wp = nn.Parameter(wt)
        bt = torch.from_numpy(b).float() 
        bp = nn.Parameter(bt)
        layer.weight = wp 
        layer.bias = bp 
        
    def read_weights_from_file(self):
        reader = FilePersistenceHelper.read_from_file("iris.multilayer")
        
        reader.push_level("layer1")
        w = reader.get_numpy_array("w")
        b = reader.get_numpy_array("b")
        self._set_weights(self.dense1, w, b)
        reader.pop_level()
        
        reader.push_level("layer2")
        w = reader.get_numpy_array("w")
        b = reader.get_numpy_array("b")
        self._set_weights(self.dense2, w, b)
        reader.pop_level()
        
        reader.push_level("layer3")
        w = reader.get_numpy_array("w")
        b = reader.get_numpy_array("b")
        self._set_weights(self.dense3, w, b)
        
    def print_weights(self):
        print("------------------  Dense 1 ---------------------")
        print("-------------------------------------------------")
        print(self.dense1.weight.data)
        print("Bias:")
        print(self.dense1.bias.data)
        print("------------------  Dense 2 ---------------------")
        print("-------------------------------------------------")
        print(self.dense2.weight.data)
        print("Bias:")
        print(self.dense2.bias.data)
        print("------------------  Dense 3 ---------------------")
        print("-------------------------------------------------")
        print(self.dense3.weight.data)
        print("Bias:")
        print(self.dense3.bias.data)

In [6]:
input = torch.FloatTensor([[1, -1, 2,1]])
print(input)
net = Net() 
net.read_weights_from_file() 
output = net(input)
print(output)
#net.print_weights()

tensor([[ 1., -1.,  2.,  1.]])
tensor([[0.3028, 0.3063, 0.3910]], grad_fn=<SoftmaxBackward>)


In [7]:
target = torch.tensor([1])
target = target.view(1, )
criteria = nn.NLLLoss()
loss = criteria(output, target)
optimizer = torch.optim.SGD(net.parameters(), lr=.01)
print(net)

Net(
  (dense1): Linear(in_features=4, out_features=16, bias=True)
  (dense2): Linear(in_features=16, out_features=10, bias=True)
  (dense3): Linear(in_features=10, out_features=3, bias=True)
  (softmax): Softmax(dim=1)
)


In [8]:
epoch = 0
total_epochs = 10000
log_at_info() 
for x, y in iris.train_iterator(total_epochs,batch_size=1, one_hot=False): 
    torch_x = torch.from_numpy(x.T).float() 
    target_y = torch.from_numpy(y).long()  
    #print("x={}, y={}".format(torch_x, target_y.data))
    predicted = net(torch_x) 
    loss = criteria(predicted, target_y )
    if epoch %500==0 : 
        print("[{}] Loss:{}, Target:{}, predicted:{}".format(epoch, loss, target_y.data, predicted.data))
    epoch+=1
    optimizer.zero_grad() 
    loss.backward() 
    optimizer.step() 
    
print("[{}] Loss:{}".format(epoch, loss))

Setting level for all handlers to INFO


[0] Loss:-0.40078672766685486, Target:tensor([2]), predicted:tensor([[0.2952, 0.3040, 0.4008]])
[500] Loss:-0.2396797239780426, Target:tensor([1]), predicted:tensor([[0.2414, 0.2397, 0.5189]])
[1000] Loss:-0.1419222205877304, Target:tensor([1]), predicted:tensor([[0.0468, 0.1419, 0.8113]])
[1500] Loss:-0.9716880321502686, Target:tensor([0]), predicted:tensor([[0.9717, 0.0267, 0.0016]])
[2000] Loss:-0.9135372638702393, Target:tensor([2]), predicted:tensor([[2.0418e-04, 8.6259e-02, 9.1354e-01]])
[2500] Loss:-0.9635754823684692, Target:tensor([2]), predicted:tensor([[7.1971e-06, 3.6417e-02, 9.6358e-01]])
[3000] Loss:-0.8674612641334534, Target:tensor([1]), predicted:tensor([[0.0017, 0.8675, 0.1308]])
[3500] Loss:-0.9890978336334229, Target:tensor([0]), predicted:tensor([[9.8910e-01, 1.0902e-02, 4.6344e-09]])
[4000] Loss:-0.9915999174118042, Target:tensor([0]), predicted:tensor([[9.9160e-01, 8.4001e-03, 7.5163e-10]])
[4500] Loss:-0.9689174294471741, Target:tensor([2]), predicted:tensor([[2

In [9]:
num_correct = 0 
test_indexes = list(iris.test_idx)
for i in test_indexes: 
    x = torch.from_numpy(iris.data[i,:].reshape(-1,4)).float() 
    y_actual = iris.targets[i] 
    y_pred = net(x) 
    max_idx = torch.argmax(y_pred) 
    mark = "x"
    if max_idx.data == y_actual : 
        mark = u'\u2713'
        num_correct+=1
    print("X:{}, Target={}, Predicted:{}  {}".format( x, y_actual, max_idx, mark))
    #print(repr(value))

        
total = len(test_indexes)
percent = num_correct*100.0/total
print(percent)

X:tensor([[5.1000, 3.5000, 1.4000, 0.2000]]), Target=0, Predicted:0  ✓
X:tensor([[4.7000, 3.2000, 1.3000, 0.2000]]), Target=0, Predicted:0  ✓
X:tensor([[7.4000, 2.8000, 6.1000, 1.9000]]), Target=2, Predicted:2  ✓
X:tensor([[5.4000, 3.9000, 1.7000, 0.4000]]), Target=0, Predicted:0  ✓
X:tensor([[6.1000, 2.6000, 5.6000, 1.4000]]), Target=2, Predicted:2  ✓
X:tensor([[6.3000, 3.4000, 5.6000, 2.4000]]), Target=2, Predicted:2  ✓
X:tensor([[6.4000, 3.1000, 5.5000, 1.8000]]), Target=2, Predicted:2  ✓
X:tensor([[4.8000, 3.4000, 1.6000, 0.2000]]), Target=0, Predicted:0  ✓
X:tensor([[5.8000, 4.0000, 1.2000, 0.2000]]), Target=0, Predicted:0  ✓
X:tensor([[5.8000, 2.7000, 5.1000, 1.9000]]), Target=2, Predicted:2  ✓
X:tensor([[6.7000, 3.3000, 5.7000, 2.5000]]), Target=2, Predicted:2  ✓
X:tensor([[5.1000, 3.5000, 1.4000, 0.3000]]), Target=0, Predicted:0  ✓
X:tensor([[6.7000, 3.0000, 5.2000, 2.3000]]), Target=2, Predicted:2  ✓
X:tensor([[5.1000, 3.8000, 1.5000, 0.3000]]), Target=0, Predicted:0  ✓
X:tens